In [ ]:
def load_csv(file_path):
    all_snapshots = []
    lines_per_snapshot = 22341
    lines_of_garbage = 6
    lines_per_section = lines_per_snapshot + lines_of_garbage
    column_names = ['p', 'c', 'w', 'k', 'u', 'v', 'x', 'y']
    snapshot_number = 0
    for chunk in pd.read_csv(file_path, 
                             chunksize=lines_per_section,
                             skip_blank_lines=False,
                             names=column_names,
                             delimiter=','):       
        snapshot_number += 1
        chunk['snapshot'] = None
        data_lines = chunk.iloc[lines_of_garbage:].copy()
        if (len(data_lines)==0):break
        print(f"Loading snapshot {snapshot_number}/100, Length: {len(data_lines)}", end='\r', flush=True)
        data_lines['t'] = snapshot_number
        all_snapshots.append(data_lines)
    all_data = pd.concat(all_snapshots, ignore_index=True)
    return all_data

In [ ]:
# Load your data using the load_csv function
data = load_csv('your_data.csv')

# Split the data into train and test sets (you can adjust the test_size)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define a custom PyTorch Dataset for your data
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.x = torch.tensor(data[['x', 'y', 't', 'wind', 'leak_x', 'leak_y', 'leak_s']].values, dtype=torch.float32)
        self.y = torch.tensor(data[['u', 'v', 'c', 'k', 'w']].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create train and test datasets
train_dataset = MyDataset(train_data)
test_dataset = MyDataset(test_data)

# Create DataLoader instances for batching
batch_size = 32  # You can adjust this based on your needs
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the neural network architecture
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(7, 20))
        self.layers.append(nn.Tanh())
        for _ in range(7):
            self.layers.append(nn.Linear(20, 20))
            self.layers.append(nn.Tanh())
        self.output_layer = nn.Linear(20, 5)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Create an instance of the neural network
net = MyNet()

# Define the loss function
criterion = nn.MSELoss()

# Dummy input data (you should replace this with your actual data)
input_data = torch.randn(1, 7, requires_grad=True)  # Requires gradients

# Forward pass through the network
outputs = net(input_data)

# Calculate derivatives element-wise
u_x = torch.autograd.grad(outputs[:, 0].sum(), input_data, create_graph=True)[0]
v_y = torch.autograd.grad(outputs[:, 1].sum(), input_data, create_graph=True)[0]
c_x = torch.autograd.grad(outputs[:, 2].sum(), input_data, create_graph=True)[0]  # Updated
c_y = torch.autograd.grad(outputs[:, 3].sum(), input_data, create_graph=True)[0]  # Updated
c_t = torch.autograd.grad(outputs[:, 4].sum(), input_data, create_graph=True)[0]  # Updated
c_xx = torch.autograd.grad(c_x[:, 0].sum(), input_data, create_graph=True)[0]  # Updated
c_yy = torch.autograd.grad(c_y[:, 1].sum(), input_data, create_graph=True)[0]  # Updated

# Calculate continuity and convection functions
continuity = u_x[:, 0] + v_y[:, 1]
convection = (outputs[:, 0] * u_x[:, 0] + outputs[:, 1] * v_y[:, 1] +
              outputs[:, 2] * c_x[:, 0] + outputs[:, 3] * c_y[:, 1] +
              outputs[:, 4] * c_t[:, 2] - c_xx[:, 0] - c_yy[:, 1])

# Define target values (you should replace this with your target data)
u_pred = torch.randn(1, 1)
v_pred = torch.randn(1, 1)
c_pred = torch.randn(1, 1)
k_pred = torch.randn(1, 1)
w_pred = torch.randn(1, 1)

# Calculate the loss
loss = criterion(outputs[:, 0], u_pred) + \
       criterion(outputs[:, 1], v_pred) + \
       criterion(outputs[:, 2], c_pred) + \
       criterion(outputs[:, 3], k_pred) + \
       criterion(outputs[:, 4], w_pred) + \
       criterion(continuity, torch.zeros_like(continuity)) + \
       criterion(convection, torch.zeros_like(convection))

# Backpropagation and optimization (you should replace this with your training loop)
optimizer = optim.Adam(net.parameters(), lr=0.001)
optimizer.zero_grad()
loss.backward()
optimizer.step()

# Print the loss
print(loss.item())


C:\Users\mzomo\AppData\Roaming\Python\Python39\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0.8990346193313599


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Define your load_csv function here
def load_csv(file_path):
    # Your implementation to load data into a Pandas DataFrame
    # ...

# Load your data using the load_csv function
data = load_csv('your_data.csv')  # Replace 'your_data.csv' with your actual data file path

# Split the data into train and test sets (you can adjust the test_size)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Define a custom PyTorch Dataset for your data
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.x = torch.tensor(data[['x', 'y', 't', 'wind', 'leak_x', 'leak_y', 'leak_s']].values, dtype=torch.float32)
        self.y = torch.tensor(data[['u', 'v', 'c', 'k', 'w']].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# Create train and test datasets
train_dataset = MyDataset(train_data)
test_dataset = MyDataset(test_data)

# Create DataLoader instances for batching
batch_size = 32  # You can adjust this based on your needs
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the neural network architecture
class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(7, 20))
        self.layers.append(nn.Tanh())
        for _ in range(7):
            self.layers.append(nn.Linear(20, 20))
            self.layers.append(nn.Tanh())
        self.output_layer = nn.Linear(20, 5)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

# Create an instance of the neural network
net = MyNet()

# Define the loss function
criterion = nn.MSELoss()

# Define the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training loop (you should replace this with your actual training loop)
epochs = 10  # You can adjust the number of epochs
for epoch in range(epochs):
    for batch in train_loader:
        inputs, targets = batch
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Evaluate on the test set (you should replace this with your evaluation code)
    with torch.no_grad():
        test_loss = 0.0
        for batch in test_loader:
            inputs, targets = batch
            outputs = net(inputs)
            test_loss += criterion(outputs, targets).item()
        print(f'Epoch {epoch + 1}/{epochs}, Test Loss: {test_loss / len(test_loader)}')

# Save your trained model if needed
# torch.save(net.state_dict(), 'my_model.pth')
